In [1]:
!pip install datasets
!pip install scikit-learn
!pip install ibm-watson-machine-learning==1.0.312

In [2]:
import os, getpass
from pandas import read_csv

In [3]:
credentials = {
    "url" : "https://eu-de.ml.cloud.ibm.com" ,
    "apikey" : getpass.getpass("JI_XLUnZsrPd4qB79c1uiJav5qj5VwkGDLyJ1LFzyXgU")
}

JI_XLUnZsrPd4qB79c1uiJav5qj5VwkGDLyJ1LFzyXgU ········


In [4]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your Project ID")

In [5]:
project_id

'd88214da-1189-4427-9d09-ba454e6f138c'

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='k37RD0s9VFtOJts778cYMTwDjxbST7_hAq_nA0xYozTp',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'project2-donotdelete-pr-zyfoykg0etwid2'
object_key = 'car_rental_train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,ID,Gender,Age,Customer Status,Customer Service,Satisfaction,Business Area
0,1,Male,23,Active,The app crashed multiple times,0,App Usability
1,2,Other,67,Active,Pricing was transparent and fair,0,Pricing
2,3,Male,60,Inactive,I had to wait too long for dropoff,0,Dropoff Process
3,4,Female,31,Active,Dropoff was fast and efficient,0,Dropoff Process
4,5,Female,50,Active,Had trouble locating the rental office,1,Pickup Process
5,6,Male,39,Active,App interface is user-friendly,0,App Usability
6,7,Female,61,Inactive,Support resolved my issue quickly,0,Customer Support
7,8,Male,69,Inactive,Pricing was transparent and fair,0,Pricing
8,9,Female,51,Inactive,Confusing dropoff instructions,0,Dropoff Process
9,10,Female,27,Inactive,Booking was seamless through the app,0,App Usability


In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='k37RD0s9VFtOJts778cYMTwDjxbST7_hAq_nA0xYozTp',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'project2-donotdelete-pr-zyfoykg0etwid2'
object_key = 'car_rental_test.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_data = pd.read_csv(body)
test_data.head(10)


,ID,Gender,Age,Customer Status,Customer Service,Satisfaction,Business Area
0,1,Female,68,Active,Staff at pickup were helpful,0,Pickup Process
1,2,Male,40,Inactive,I received an old model with scratches,0,Vehicle Quality
2,3,Male,33,Inactive,Confusing dropoff instructions,0,Dropoff Process
3,4,Male,44,Active,Dropoff was fast and efficient,1,Dropoff Process
4,5,Male,61,Inactive,Difficult to navigate the app,1,App Usability
5,6,Female,66,Inactive,The vehicle had mechanical issues,0,Vehicle Quality
6,7,Other,48,Active,Customer support was very helpful,0,Customer Support
7,8,Other,64,Inactive,No one responded to my issue,0,Customer Support
8,9,Other,40,Active,The rental price was too high,1,Pricing
9,10,Female,27,Inactive,Dropoff was fast and efficient,1,Dropoff Process


In [8]:
df_1.shape

(500, 7)

In [9]:
test_data.shape

(10, 7)

In [10]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [11]:
model_id = ModelTypes.FLAN_UL2

In [12]:
satisfaction_instruction = """
Was customer satisfied?\n

comment: I have had a few recent rentals that have taken a very long time , 
with no offer of apology.
In the most recent case , the agent subsequently offered me a car type on
an upgrade coupon and then told me it was no longer available\n

satisfaction :0\n\n
"""

In [13]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.MAX_NEW_TOKENS: 10
}

In [14]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id = model_id,
    params = parameters,
    credentials = credentials,
    project_id=project_id)

In [15]:
results = []
comments = list(test_data["Customer Service"])
satisfaction = test_data["Satisfaction"].astype(str).tolist()

for input_text in comments:
    results.append(model.generate_text(prompt=" ".join([satisfaction_instruction,input_text])))

In [16]:
comments

['Staff at pickup were helpful',
 'I received an old model with scratches',
 'Confusing dropoff instructions',
 'Dropoff was fast and efficient',
 'Difficult to navigate the app',
 'The vehicle had mechanical issues',
 'Customer support was very helpful',
 'No one responded to my issue',
 'The rental price was too high',
 'Dropoff was fast and efficient']

In [17]:
satisfaction

['0', '0', '0', '1', '1', '0', '0', '0', '1', '1']

In [18]:
results

['no', 'no', 'no', 'no', 'no', 'no', 'satisfaction :1', 'No', 'no', 'no']

In [19]:
business_area_instruction = """
Find the business area of the customer email.
Choose business area from the following list:
'App Usability' , 'Pricing' , 'Dropoff Process' , 
'Pickup Process' , 'Customer Support' , 'Vehicle Quality ' \n

comment : I do not understand why I have to pay additional fee if vehicle is 
returned without a full tank. \n
business area : 'Pricing' \n\n

"""

In [20]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.MAX_NEW_TOKENS: 15
}

In [21]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id = model_id,
    params = parameters,
    credentials = credentials,
    project_id=project_id)

In [22]:
results = []
comments = list(test_data["Customer Service"])
area = list(test_data["Business Area"])

for input_text in comments:
    results.append(model.generate_text(prompt=" ".join([business_area_instruction,input_text])))

In [23]:
comments

['Staff at pickup were helpful',
 'I received an old model with scratches',
 'Confusing dropoff instructions',
 'Dropoff was fast and efficient',
 'Difficult to navigate the app',
 'The vehicle had mechanical issues',
 'Customer support was very helpful',
 'No one responded to my issue',
 'The rental price was too high',
 'Dropoff was fast and efficient']

In [24]:
area

['Pickup Process',
 'Vehicle Quality',
 'Dropoff Process',
 'Dropoff Process',
 'App Usability',
 'Vehicle Quality',
 'Customer Support',
 'Customer Support',
 'Pricing',
 'Dropoff Process']

In [25]:
results

['Pickup Process',
 'Vehicle Quality',
 'Dropoff Process',
 'Dropoff Process',
 'App Usability',
 'Vehicle Quality',
 'Customer Support',
 'Customer Support',
 'Pricing',
 'Dropoff Process']